In [2]:
import pandas as pd
import datetime
import numpy as np
import scipy.stats
from math import log10 , floor

# file 불러오기

In [3]:
# volume data 가져오기
df_vol = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/volume_data.xlsx', header=[0,1], index_col=0, engine='openpyxl')
df_vol.columns.names = ["ticker",'variable']
# 구조: ticker - name - variable
# 거래량(주)	평균발행주식수 (보통)(주)	매수수량(기관계)(주)	매도수량(기관계)(주)	매수수량(개인)(주)	매도수량(개인)(주)	매수수량(외국인계)(주)	매도수량(외국인계)(주)
df_vol.index = pd.to_datetime(df_vol.index) # 날짜 형식 변경

In [4]:
# value(거래대금) data 가져오기
# 거래대금(원)	매수대금(기관계)(만원)	매도대금(기관계)(만원)	매수대금(개인)(만원)	매도대금(개인)(만원)	매수대금(외국인계)(만원)	매도대금(외국인계)(만원)
df_val = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/volume_value_data.xlsx', header=[0,1], index_col=0, engine='openpyxl')
df_val.columns.names = ["ticker",'variable']
df_val.index = pd.to_datetime(df_val.index)     # 날짜 형식 변경

KeyboardInterrupt: 

In [ ]:
# 두 df 합치기
df_vol = pd.concat([df_vol, df_val], axis=1)
df_vol.sort_index(axis=1, inplace=True)         # col index 정렬 -> loc 오류 방지

## refine 전 데이터 사용 : df 

In [ ]:
# result 구분
df = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AR.xlsx', engine='openpyxl') #일자, 기업
# 날짜 형식 다름
df['일자'] = df['일자'].astype(str)
df['일자'].str[-8:]
df['일자'] = pd.to_datetime(df['일자'])

In [ ]:
df.sort_values(by=['일자','기업'], ascending=[False, True] ,inplace=True)

In [ ]:
df = df.loc[:,['일자','제목','기업','ESG','중복 횟수']]

In [ ]:
df.reset_index(inplace =True, drop=True)

In [ ]:
df

In [10]:
df['중복 횟수'].value_counts().loc[:5]

0    4157
1    1003
2     482
3     298
4     192
5     168
Name: 중복 횟수, dtype: int64

같은 날짜의 다른 ESG 이슈 -> 제거

### 같은 윈도우에 다른 키워드가 겹치는 경우 : df -> df2

In [ ]:
# 정렬
df.sort_values(by=['기업','ESG','일자'], ascending=[True,True,False] ,inplace=True)
df.reset_index(drop=True, inplace=True) # index 초기화

In [ ]:
df['save_kill'] = 'save'

i = 0
while i < df.shape[0] :
    for j in range(1,min(10,df.shape[0]-i)) :
        if (df.iloc[i]['일자'] <= df.iloc[i+j]['일자'] <= df.iloc[i]['일자'] + datetime.timedelta(days=10)) \
        & (df.iloc[i]['기업'] == df.iloc[i+j]['기업']) :
            if df.iloc[i]['중복 횟수'] > df.iloc[i+j]['중복 횟수']:
                df.loc[i, 'save_kill'] = 'save'
                df.loc[i+j, 'save_kill'] = 'kill'
            elif df.iloc[i]['중복 횟수'] < df.iloc[i+j]['중복 횟수']:
                df.loc[i, 'save_kill'] = 'kill'
                df.loc[i+j, 'save_kill'] = 'save'
            else: # 동일
                df.loc[i, 'save_kill'] = 'kill'
                df.loc[i+j, 'save_kill'] = 'kill'
    i +=1

    if i%600 == 0  :
        print(i)

df2 = df[df['save_kill']=='save']


In [ ]:
df2.reset_index(drop=True, inplace=True) 
df2

## 실적 발표 제거: df2 -> df3

In [ ]:
df_anc = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/anounce.xlsx', engine='openpyxl') 
df_anc['접수일자'] = pd.to_datetime(df_anc['접수일자'])

df_kospi = pd.read_csv('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/data/KOSPI.csv',  encoding='cp949')
df_kospi['종목코드'] = df_kospi['종목코드'].astype(str).str.zfill(6) 
ticker_dict = dict(zip(df_kospi['종목명'].to_list(),df_kospi['종목코드'].to_list())) # 종목명이 key
ticker_dict2 = dict(zip(df_kospi['종목코드'].to_list(),df_kospi['종목명'].to_list())) 

df2['일자'] = pd.to_datetime(df2['일자'])
column = list(df2.columns)
df_tot = pd.DataFrame(columns=column)

stocks_name = df2['기업'].value_counts().index.to_list()
stocks = [str(ticker_dict.get(i)) for i in stocks_name]

for stock in stocks_name:
    df_anc_temp = df_anc[df_anc['종목명']==stock]
    df2_temp = df2[df2['기업']==stock]
    elim = []
    for i in range(df2_temp.shape[0]):
        elim_temp = []
        for j in range(df_anc_temp.shape[0]):
            if (df2_temp.iloc[i]['일자'] - datetime.timedelta(days=10) <= df_anc_temp.iloc[j]['접수일자'] <= df2_temp.iloc[i]['일자'] + datetime.timedelta(days=10)) :
                elim_temp.append(True)
            else:
                elim_temp.append(False)
        if True in elim_temp : # 하나라도 T가 있으면 중복되는 기사가 있으므로 T로  저장
            elim.append(True)
        else : # 전부 F
            elim.append(False)
    df2_temp['제거']=elim
    df2_temp = df2_temp[df2_temp['제거']==False]
    df_tot = pd.concat([df_tot, df2_temp], ignore_index=True)
    

In [ ]:
df_tot

In [ ]:
df3 = df_tot.copy()

# variable 정의

## Abnormal Volume
Vol.1 $v(i,t)*v(m)/v(i)*v(m,t)$ \
Vol.2 $v(i,t)/v(m,t) - v(i)/v(m)$ \
Vol.3 $v(i,t)/v(i) - v(mt)/v(m)$

In [ ]:
# v_i, v_m : 영업일 기준 200일(-205~-6일) => initial 값에 의존하므로 initial의 함수로 처리

def AV_tot(vol, df_data, date):

    # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산
    a = ind+initial-period
    b = ind+initial-1
    v_i = df_data.loc[a:b, [vol]].mean()
    v_m = df_kospi_vol.loc[a:b, [vol]].mean()

    seq = [] # -5일부터 +5일까지 11개의 av
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음)
            seq.append(np.nan) 
        else:
            v_it = df_data.iloc[ind+j][vol]
            v_mt = df_kospi_vol.iloc[ind_mkt+j][vol]
            av = v_it/v_i - v_mt/v_m 
#             av = (v_it*v_m)/(v_mt*v_i)
            seq.append(av[0])
            
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정
    return seq_2

In [ ]:
def AV_inv(vol, buy, sell, df_data, date):

   # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산
    a = ind+initial-period
    b = ind+initial-1
    v_i = df_data.loc[a:b,[buy,sell]].mean(axis=1).mean(axis=0)
    v_m = df_kospi_vol.loc[a:b, [vol]].mean()
    
    seq = [] # -5일부터 +5일까지 11개의 av
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음) nan으로 처리 -> 나중에 제거
            seq.append(np.nan) 
        else:
            ### null 제어하면서 계산 ###
            if np.isnan(df_data.iloc[ind+j][buy])  :    # 아무것도 없으면 0으로 처리 => 0(수식상) vs 1(개념상)
                seq.append(0)
                
            else:  
                v_it = (df_data.iloc[ind+j][buy]+df_data.iloc[ind+j][sell])/2
                v_mt = df_kospi_vol.iloc[ind_mkt+j][vol]
                av = v_it/v_i - v_mt/v_m
#                 av = (v_it*v_m)/(v_mt*v_i)
                seq.append(av[0])
            
            
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정

    return seq_2


## BSI

In [ ]:
def BSI(buy, sell, df_data, date): #, nan_mode

    seq = [] # -5일부터 +5일까지 11개의 bsi
    for j in range(initial, final+1):               # -5~5까지 각 날짜의 bsi 계산하는 loop

        df_temp = df_data.shift(-j)             #minus는 방향 보정

        buy_vol = df_temp.loc[date,buy]         # 해당 종목의 매수수량
        sell_vol = df_temp.loc[date,sell]       # 해당 종목의 매도수량
        
        ### null 제어하면서 BSI 계산 ###
        # 나눌 수 없는 경우: np.nan이거나 0, 0 인 경우이므로 제거
        # 상장되지 않았을 때의 사건은 제거된 상황
        if (buy_vol==0 and sell_vol==0) or (np.isnan(buy_vol) or np.isnan(sell_vol)) :    # 하나라도 nan이거나 둘다 0인 경우
            #############
            # 0 or np.nan?
#             if nan_mode:
#                 bsi = np.nan
#             else: 
            bsi = 0      
            #############
        else: 
            bsi = buy_vol/(buy_vol+sell_vol) -0.5
        seq.append(bsi)
        
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음
    
    return seq_2
    

## TurnOver

In [ ]:

def TO_tot(vol, df_data, date):

    # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산 (vol/out을 각각 200일 각각)
    to_mean = []
    for j in range(ind+initial-period,ind+initial):
        to_past = df_data.loc[j][vol] / df_data.loc[j]['평균발행주식수 (보통)(주)']
        to_mean.append(to_past)
    to_mean = np.array(to_mean) 
    to_i = to_mean.mean()
    
    # -5일부터 +5일까지 11개의 to
    seq = [] 
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음)
            seq.append(np.nan) 
        else:
            v_it = df_data.loc[ind+j][vol]          
            o_it = df_data.loc[ind+j]['평균발행주식수 (보통)(주)']          
            to = v_it/o_it - to_i
            seq.append(to)         
            
    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정
        
    return seq_2


In [ ]:

def TO_inv(vol, buy, sell, df_data, date):

    # 주의) 이하 index는 date -> 숫자 로 변경됨
    df_data.index.name = 'date'             # index 이름 변경
    df_data.reset_index(inplace=True)       # index 재설정
    ind_mkt = df_kospi_vol.index[(df_data['date']==date)] # df_kospi_vol의 해당 날짜의 index 넘버
    ind = df_data.index[(df_data['date']==date)] # df_data의 해당 날짜의 index 넘버
    ind = ind[0]
    ind_mkt = ind_mkt[0]
    
    #200일 평균값 계산 (vol/out을 각각 200일 각각)
    to_mean = []
    for j in range(ind+initial-period,ind+initial):
        if df_data.loc[j][buy] == np.nan: # or df_data.loc[j][sell] == np.nan
            to_past = 0
        else:
            to_past = (df_data.loc[j][buy] + df_data.loc[j][sell]) / (2*df_data.loc[j]['평균발행주식수 (보통)(주)'])
        to_mean.append(to_past)
    to_mean = np.array(to_mean)    
    to_i = to_mean.mean()
    
    # -5일부터 +5일까지 11개의 to
    seq = [] 
    for j in range(initial, final+1):       # -5~5까지 각 날짜의 bsi 계산하는 loop
        if np.isnan(df_data.iloc[ind-initial-period][vol]): # period가 부족한 경우(상장일이 늦음)
            seq.append(np.nan) 
        else:
            v_it = (df_data.loc[ind+j][buy] + df_data.loc[ind+j][sell])/2
            o_it = df_data.loc[ind+j]['평균발행주식수 (보통)(주)']          
            to = v_it/o_it - to_i
            seq.append(to)         

    seq_2 = []
    for a,b in window:     # 6종의 각 평균값
        seq_2.append(np.mean(seq[a-initial : b-initial+1]))   # [-n,-n+1,...,m] -> [0,1,...,n+m] 보정 (차이 = |-n|), 각 window에 대한 평균값을 원소로 넣음

    # index 원래대로
    df_data.set_index('date', inplace=True)       # index 재설정
        
    return seq_2

## var. 연산

In [25]:
# kospi volume data 가져오기
# date - ror - vol
df_kospi_vol = pd.read_csv('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/data/KOSPI_vol.csv', encoding='cp949')
df_kospi_vol['date'] = pd.to_datetime(df_kospi_vol['date']) # 날짜 형식 변경

In [26]:
# 21년12월 말은 제거
df3 = df3[df3['일자'] < datetime.datetime.strptime('2021-12-20','%Y-%m-%d')]
df3 = df3[df3['기업'] != '맥쿼리인프라']

In [27]:
df3.sort_values(by=['기업','ESG','일자'], ascending=[True,True,False] ,inplace=True)
df3.reset_index(inplace=True, drop=True)
df3

,일자,제목,기업,ESG,중복 횟수,save_kill,제거
0,2018-06-04,"BGF, SK '헬로네이처' 경영권 인수 온라인 신선식품 출사표",BGF,경영권,0,save,False
1,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save,False
2,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save,False
3,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save,False
4,2017-05-01,"BNK금융지주 시세조종, 성세환 회장 등 9명 기소",BNK금융지주,불공정거래,2,save,False
...,...,...,...,...,...,...,...
5181,2017-07-12,면세점 비리 의혹에 한화갤러리아 '울고' 호텔신라 '웃고',호텔신라,비리,0,save,False
5182,2017-06-09,"임우재 뇌물 혐의, 이부진의 호텔신라에 ‘불똥’",호텔신라,비리,1,save,False
5183,2019-09-10,"효성중공업, 국세청 382억원 추징금 부과",효성중공업,공급망,0,save,False
5184,2021-04-16,효성첨단소재 공장서 황산 1천ℓ 누출 “인명 피해 없어”,효성첨단소재,산재,1,save,False


### wide window

In [39]:
# window = [(-5,-4),(-3,-2),(-1,1),(2,3),(4,5)]
window = [(-20,-11),(-10,-5),(-5,-2),(-1,-1),(0,0),(1,1),(2,5),(6,10),(11,20)]
initial = -20 
final = 20
period = 200 # 평균값을 계산하는 기간

In [48]:
df_kospi['종목코드'] = df_kospi['종목코드'].astype(str).str.zfill(6) 
name_ticker = dict(zip(df_kospi['종목명'].to_list(),df_kospi['종목코드'].to_list())) # {종목명:tikcer}
ticker_name = dict(zip(df_kospi['종목코드'].to_list(),df_kospi['종목명'].to_list())) # {ticker:종목명}


# columns 이름 설정
columns = []
ms = ['AV_VOL', 'AV_VAL','BSI','TO']
invs = ['TOT','IND', 'INS' ,'FOR'] 
col_temp = []
for m in ms:
    for inv in invs:
        col_temp.append(m+'_'+inv)
col_temp.remove('BSI_TOT')

for c in col_temp:
    for a,b in window:
        columns.append(c+'_'+str(a)+'_'+str(b))

# +20일까지라 더 많이 잘라야함
df3_day = df3[df3['일자'] < datetime.datetime.strptime('2021-10-26','%Y-%m-%d')]
# df3_day = df3_day[df3_day['일자'] < datetime.datetime.strptime('2021-11-20','%Y-%m-%d')]

tot = [] # 각 사건의 window별 계산결과
# 개별 사건에 대한 BSI 계산
for i in range(df3_day.shape[0]) : #df.shape[0]
    ticker = name_ticker.get(df3_day.iloc[i]['기업'])    # 해당 기업의 종목코드
    date = df3_day.iloc[i]['일자']                       # 해당 사건의 날짜
    
    if not (date in df_vol.index): # 해당날짜가 비영업일인 경우 다음 영업일을 계산 (vol/val 상관없음)
        while not(date in df_vol.index): # False일 때에만 작동
            date += datetime.timedelta(days=1)
            if date > datetime.datetime.strptime('2021-12-31','%Y-%m-%d'): #날짜 넘어가면 제거
                break
    
    df_data = df_vol.loc[:,(ticker)]        # 해당 ticker의 모든 var.에 대한 자료를 별도의 df에 저장 (single-index)        # 주의) 매도가 먼저 나옴


    window_av_tot_vol = AV_tot('거래량(주)', df_data,  date)
    window_av_ind_vol = AV_inv('거래량(주)','매수수량(개인)(주)','매도수량(개인)(주)',df_data,  date)
    window_av_ins_vol = AV_inv('거래량(주)','매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_av_for_vol = AV_inv('거래량(주)','매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_av_tot_val = AV_tot('거래대금(원)', df_data,  date)
    window_av_ind_val = AV_inv('거래대금(원)','매수대금(개인)(만원)','매도대금(개인)(만원)', df_data,  date)
    window_av_ins_val = AV_inv('거래대금(원)','매수대금(기관계)(만원)',	'매도대금(기관계)(만원)', df_data,  date)
    window_av_for_val = AV_inv('거래대금(원)','매수대금(외국인계)(만원)', '매도대금(외국인계)(만원)',df_data,  date)

    window_bsi_ind = BSI('매수수량(개인)(주)','매도수량(개인)(주)', df_data,  date)
    window_bsi_ins = BSI('매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_bsi_for = BSI('매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_to_tot = TO_tot('거래량(주)', df_data,  date)
    window_to_ind = TO_inv('거래량(주)','매수수량(개인)(주)','매도수량(개인)(주)', df_data,  date)
    window_to_ins = TO_inv('거래량(주)','매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_to_for = TO_inv('거래량(주)','매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_tot = window_av_tot_vol + window_av_ind_vol + window_av_ins_vol + window_av_for_vol \
        + window_av_tot_val + window_av_ind_val + window_av_ins_val + window_av_for_val \
            + window_bsi_ind + window_bsi_ins + window_bsi_for \
                + window_to_tot + window_to_ind + window_to_ins + window_to_for

    tot.append(window_tot)  # 이중리스트
    
    if i%100 == 0:
        print(round(i/df3_day.shape[0],3))
    
tot = np.array(tot)
df_temp = pd.DataFrame(tot, columns=columns)
df3_day = pd.concat([df3_day, df_temp], axis=1)

0.0
0.019
0.039
0.058
0.078
0.097
0.117
0.136
0.156
0.175
0.195
0.214
0.234
0.253
0.273
0.292
0.312
0.331
0.351
0.37
0.389
0.409
0.428
0.448
0.467
0.487
0.506
0.526
0.545
0.565
0.584
0.604
0.623
0.643
0.662
0.682
0.701
0.721
0.74
0.759
0.779
0.798
0.818
0.837
0.857
0.876
0.896
0.915
0.935
0.954
0.974
0.993


In [50]:
df3_day.dropna(inplace=True)
df3_day.reset_index(inplace=True,drop=True)
df3_day

,일자,제목,기업,ESG,중복 횟수,save_kill,제거,AV_VOL_TOT_-20_-11,AV_VOL_TOT_-10_-5,AV_VOL_TOT_-5_-2,...,TO_INS_11_20,TO_FOR_-20_-11,TO_FOR_-10_-5,TO_FOR_-5_-2,TO_FOR_-1_-1,TO_FOR_0_0,TO_FOR_1_1,TO_FOR_2_5,TO_FOR_6_10,TO_FOR_11_20
0,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save,False,0.022955,0.453751,0.147543,...,0.000505,0.000025,0.000686,-0.000240,-0.000581,-0.000713,-0.000738,-0.000059,0.000711,0.009377
1,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save,False,-0.068276,0.301969,-0.171517,...,0.000175,-0.000326,-0.000519,-0.000438,0.000235,-0.000511,-0.000630,0.000089,0.000240,-0.000041
2,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save,False,0.333392,-0.021673,0.039662,...,-0.000086,0.000454,0.000344,-0.000004,-0.000020,-0.000402,0.000890,0.000602,0.000790,0.000422
3,2017-05-01,"BNK금융지주 시세조종, 성세환 회장 등 9명 기소",BNK금융지주,불공정거래,2,save,False,-0.379932,0.208368,0.399766,...,0.000084,0.000365,0.000227,0.000434,0.000502,-0.000001,0.000171,0.000865,0.000375,0.000292
4,2017-04-19,'주가 조작 혐의' 성세환 BNK금융지주 회장 구속,BNK금융지주,불공정거래,1,save,False,-0.220359,-0.336576,-0.216455,...,-0.000060,0.000480,0.000644,0.000026,-0.000361,0.000856,0.000322,0.000329,0.000258,0.000513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,2017-04-24,"시민단체, `늑장 리콜 의혹` 현대차 사기 혐의로 고발",현대차,품질,0,save,False,-0.606630,-0.316452,0.456815,...,-0.001072,-0.002034,0.000200,0.002816,-0.000308,0.001966,-0.001722,-0.000030,0.001435,-0.001254
4273,2017-04-21,"현대차, ""세타2 엔진결함 고발직원 복직은 불가하다""",현대차,품질,0,save,False,0.046552,-0.434879,-0.276064,...,0.000510,0.000043,-0.001124,-0.001096,-0.001655,0.003977,0.001041,0.000793,0.001233,0.000366
4274,2017-04-11,현대차 136만대 리콜에 이어 제네시스 에쿠스에서도 결함 발견돼,현대차,품질,3,save,False,0.554047,-0.350910,-0.202590,...,-0.000709,0.000893,-0.000448,-0.000348,0.002102,-0.000029,-0.000739,0.001557,0.000137,-0.001052
4275,2017-04-10,문제없다더니 현대차 세타 엔진 국내서도 결국 리콜,현대차,품질,1,save,False,-0.088066,-0.200059,-0.353899,...,-0.001114,0.000109,-0.000146,-0.000385,-0.000366,-0.000401,-0.000025,-0.000298,-0.000246,-0.000321


In [51]:
df3_day.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_result_6.xlsx', index=None)
# result 6: window -20~+20, no +-10 filtering

### window2

In [297]:
# window = [(-5,-4),(-3,-2),(-1,1),(2,3),(4,5)]
window = [(-5,-2),(-1,-1),(0,0),(1,1),(2,5)]
initial = -5 
final = 5
period = 200 # 평균값을 계산하는 기간

In [298]:
df_kospi['종목코드'] = df_kospi['종목코드'].astype(str).str.zfill(6) 
name_ticker = dict(zip(df_kospi['종목명'].to_list(),df_kospi['종목코드'].to_list())) # {종목명:tikcer}
ticker_name = dict(zip(df_kospi['종목코드'].to_list(),df_kospi['종목명'].to_list())) # {ticker:종목명}


# columns 이름 설정
columns = []
ms = ['AV_VOL', 'AV_VAL','BSI','TO']
invs = ['TOT','IND', 'INS' ,'FOR'] 
col_temp = []
for m in ms:
    for inv in invs:
        col_temp.append(m+'_'+inv)
col_temp.remove('BSI_TOT')

for c in col_temp:
    for a,b in window:
        columns.append(c+'_'+str(a)+'_'+str(b))

tot = [] # 각 사건의 window별 계산결과
# 개별 사건에 대한 BSI 계산
for i in range(df3.shape[0]) : #df.shape[0]
    ticker = name_ticker.get(df3.iloc[i]['기업'])    # 해당 기업의 종목코드
    date = df3.iloc[i]['일자']                       # 해당 사건의 날짜
    
    if not (date in df_vol.index): # 해당날짜가 비영업일인 경우 다음 영업일을 계산 (vol/val 상관없음)
        while not(date in df_vol.index): # False일 때에만 작동
            date += datetime.timedelta(days=1)
            if date > datetime.datetime.strptime('2021-12-31','%Y-%m-%d'): #날짜 넘어가면 제거
                break
            
    
    df_data = df_vol.loc[:,(ticker)]        # 해당 ticker의 모든 var.에 대한 자료를 별도의 df에 저장 (single-index)        # 주의) 매도가 먼저 나옴


    window_av_tot_vol = AV_tot('거래량(주)', df_data,  date)
    window_av_ind_vol = AV_inv('거래량(주)','매수수량(개인)(주)','매도수량(개인)(주)',df_data,  date)
    window_av_ins_vol = AV_inv('거래량(주)','매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_av_for_vol = AV_inv('거래량(주)','매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_av_tot_val = AV_tot('거래대금(원)', df_data,  date)
    window_av_ind_val = AV_inv('거래대금(원)','매수대금(개인)(만원)','매도대금(개인)(만원)', df_data,  date)
    window_av_ins_val = AV_inv('거래대금(원)','매수대금(기관계)(만원)',	'매도대금(기관계)(만원)', df_data,  date)
    window_av_for_val = AV_inv('거래대금(원)','매수대금(외국인계)(만원)', '매도대금(외국인계)(만원)',df_data,  date)

    window_bsi_ind = BSI('매수수량(개인)(주)','매도수량(개인)(주)', df_data,  date)
    window_bsi_ins = BSI('매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_bsi_for = BSI('매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_to_tot = TO_tot('거래량(주)', df_data,  date)
    window_to_ind = TO_inv('거래량(주)','매수수량(개인)(주)','매도수량(개인)(주)', df_data,  date)
    window_to_ins = TO_inv('거래량(주)','매수수량(기관계)(주)',	'매도수량(기관계)(주)', df_data,  date)
    window_to_for = TO_inv('거래량(주)','매수수량(외국인계)(주)', '매도수량(외국인계)(주)', df_data,  date)

    window_tot = window_av_tot_vol + window_av_ind_vol + window_av_ins_vol + window_av_for_vol \
        + window_av_tot_val + window_av_ind_val + window_av_ins_val + window_av_for_val \
            + window_bsi_ind + window_bsi_ins + window_bsi_for \
                + window_to_tot + window_to_ind + window_to_ins + window_to_for

    tot.append(window_tot)  # 이중리스트
    
    if i%300 == 0:
        print(round(i/df3.shape[0],3))
    
tot = np.array(tot)
df_temp = pd.DataFrame(tot, columns=columns)
df3 = pd.concat([df3, df_temp], axis=1)

0.0


KeyboardInterrupt: 

In [ ]:
df3

In [ ]:
df3.dropna(inplace=True)
df3.reset_index(inplace=True,drop=True)

In [144]:
df3

,일자,제목,기업,ESG,중복 횟수,save_kill,제거,AV_VOL_TOT_-5_-2,AV_VOL_TOT_-1_-1,AV_VOL_TOT_0_0,...,TO_INS_-5_-2,TO_INS_-1_-1,TO_INS_0_0,TO_INS_1_1,TO_INS_2_5,TO_FOR_-5_-2,TO_FOR_-1_-1,TO_FOR_0_0,TO_FOR_1_1,TO_FOR_2_5
0,2020-11-04,"BNK금융지주 전 간부, 주가 조종 혐의 벌금형",BNK금융지주,불공정거래,0,save,False,0.128097,-0.260732,-0.428205,...,-0.000249,-2.865339e-04,-0.000151,-0.000013,0.000053,-0.000307,-0.000648,-0.000780,-0.000805,-0.000126
1,2018-01-09,‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월,BNK금융지주,불공정거래,0,save,False,-0.188109,0.209962,-0.251972,...,-0.000180,-3.931712e-07,0.000044,0.000150,0.000396,-0.000444,0.000229,-0.000518,-0.000636,0.000083
2,2017-05-28,"BNK금융지주, 주가조작 의심세력 검찰에 고소",BNK금융지주,불공정거래,2,save,False,0.017436,0.168691,0.034991,...,0.000085,4.259276e-04,0.000163,0.000183,-0.000137,-0.000033,-0.000048,-0.000430,0.000862,0.000574
3,2017-05-01,"BNK금융지주 시세조종, 성세환 회장 등 9명 기소",BNK금융지주,불공정거래,2,save,False,0.391756,0.256898,-0.037706,...,0.000154,-1.852024e-04,-0.000287,-0.000212,0.000299,0.000415,0.000483,-0.000020,0.000152,0.000846
4,2017-04-19,'주가 조작 혐의' 성세환 BNK금융지주 회장 구속,BNK금융지주,불공정거래,1,save,False,-0.208828,-0.323645,0.601944,...,-0.000345,-4.478847e-04,0.000153,-0.000116,0.000117,0.000010,-0.000377,0.000841,0.000307,0.000314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,2021-01-18,'국정농단' 이재용 징역 2년6월 법정구속 호텔신라 ↑,호텔신라,비리,0,save,False,0.595515,-0.177064,3.893270,...,0.003321,1.187790e-04,0.005161,-0.000318,0.000487,0.003211,0.000088,0.002361,-0.001326,0.000366
4425,2017-07-12,면세점 비리 의혹에 한화갤러리아 '울고' 호텔신라 '웃고',호텔신라,비리,0,save,False,-0.284601,-0.310572,1.536280,...,-0.000895,-2.711907e-04,0.001246,0.000497,0.000853,-0.001097,-0.001655,0.003976,0.001040,0.000792
4426,2017-06-09,"임우재 뇌물 혐의, 이부진의 호텔신라에 ‘불똥’",호텔신라,비리,1,save,False,-0.208108,-0.005533,-0.542903,...,-0.000786,2.695719e-04,-0.000468,-0.000535,0.000774,-0.000371,0.002079,-0.000052,-0.000761,0.001535
4427,2019-09-10,"효성중공업, 국세청 382억원 추징금 부과",효성중공업,공급망,0,save,False,-0.332025,-0.646029,-0.715219,...,-0.000786,-4.674622e-04,-0.000741,-0.000078,-0.000531,-0.000383,-0.000365,-0.000400,-0.000024,-0.000297


In [134]:
df3['기업'].value_counts().shape

(198,)

In [145]:
df3.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_result_5.xlsx', index=None)
# result 5: window -5~+5, no +-10 filtering

# t-test

In [5]:
### result 6 ###
df3 = pd.read_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_result_6.xlsx', engine='openpyxl')

## overlap 40일 제어

In [6]:
import datetime as dt
# 정렬
df3.sort_values(by=['기업','ESG','일자'], ascending=[True,True,False] ,inplace=True)
df3.reset_index(drop=True, inplace=True) # index 초기화

overlap = []
for i in range(df3.shape[0]):
    overlap_temp = [] # 각 i마다 iter에 사용
    if i < df3.shape[0]-40 :
        for j in range(1,40) :
            if (df3.iloc[i]['일자'] - dt.timedelta(days=40) <= df3.iloc[i+j]['일자'] < df3.iloc[i]['일자']) & (df3.iloc[i]['ESG'] == df3.iloc[i+j]['ESG']) & (df3.iloc[i]['기업'] == df3.iloc[i+j]['기업']) :
                overlap_temp.append(True) 
            else :
                overlap_temp.append(False)
        if True in overlap_temp : # 하나라도 T가 있으면 중복되는 기사가 있으므로 T로 overlap에 저장
            overlap.append(True)
        else : # 전부 F
            overlap.append(False)
        
    else :
        j = 1
        while i+j < df3.shape[0] :
            if (df3.iloc[i]['일자'] - dt.timedelta(days=40) <= df3.iloc[i+j]['일자'] < df3.iloc[i]['일자']) & (df3.iloc[i]['ESG'] == df3.iloc[i+j]['ESG']) & (df3.iloc[i]['기업'] == df3.iloc[i+j]['기업']) :
                overlap_temp.append(True)
            else :
                overlap_temp.append(False)
            j +=1 
        if True in overlap_temp : 
            overlap.append(True)
        else :
            overlap.append(False)
    
    if i%100 == 0:
        print(i)

df3['OVERLAP'] = overlap # T/F

# overlap 횟수
df3.sort_values(by=['기업','ESG','일자'], ascending=[True,True,True] ,inplace=True)
df3.reset_index(drop=True, inplace=True)

over_nums =[]
for i in range(df3.shape[0]):
    over_num = df3.iloc[i]['중복 횟수']
    if i < df3.shape[0]-40 :
        for j in range(1,40) :
            if (df3.iloc[i]['일자'] < df3.iloc[i+j]['일자'] <= df3.iloc[i]['일자'] + dt.timedelta(days=40)) & (df3.iloc[i]['ESG'] == df3.iloc[i+j]['ESG']) \
                & (df3.iloc[i]['기업'] == df3.iloc[i+j]['기업']) :
                over_num += df3.iloc[i+j]['중복 횟수']+1 # 해당 row도 포함
    else :
        j = 1
        while i+j < df3.shape[0] :
            if (df3.iloc[i]['일자'] < df3.iloc[i+j]['일자'] <= df3.iloc[i]['일자'] + dt.timedelta(days=40))  & (df3.iloc[i]['ESG'] == df3.iloc[i+j]['ESG']) \
                & (df3.iloc[i]['기업'] == df3.iloc[i+j]['기업']) :
                over_num += df3.iloc[i+j]['중복 횟수']+1 # 해당 row도 포함
            j +=1 
    over_nums.append(over_num)
    
    if i%100 == 0:
        print(i)
df3['중복 횟수(10일)'] = over_nums

# True 제거
df3 = df3[df3['OVERLAP']==False]

print(df3.head())
print(df3.shape)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
           일자                                   제목       기업    ESG  중복 횟수  \
0  2017-03-07               ‘시세조종’ 혐의 BNK금융지주 압수수색  BNK금융지주  불공정거래      5   
8  2018-01-09   ‘주식 시세조종’ 성세환 전 BNK금융지주 회장 징역 1년6월  BNK금융지주  불공정거래      0   
9  2020-11-04           BNK금융지주 전 간부, 주가 조종 혐의 벌금형  BNK금융지주  불공정거래      0   
10 2017-04-14  성세환 BNK금융지주 회장 등 최고위 경영진 3명 구속영장 청구  BNK금융지주     비리      1   
11 2018-03-01    '채용비리 의혹' BNK금융지주 사장 등 2명 구속영장 청구  BNK금융지주     비리      3   

   save_kill     제거  AV_VOL_TOT_-20_-11  AV_VOL_TOT_-10_-5  AV_VOL_TOT_-5_-2  \
0       save  False            0.579515           0

In [7]:

##########
# 같은 윈도우에 다른 키워드가 겹치는 경우
#########

# 정렬
df3.sort_values(by=['기업','일자'], ascending=[True,True] ,inplace=True)
df3.reset_index(drop=True, inplace=True) # index 초기화

df3['save_kill'] = 'save'
for i in range(df3.shape[0]):
    if i < df3.shape[0]-50 :
        for j in range(1,50) :
            if (df3.iloc[i]['일자'] <= df3.iloc[i+j]['일자'] <= df3.iloc[i]['일자'] + dt.timedelta(days=40)) & (df3.iloc[i]['기업'] == df3.iloc[i+j]['기업']) :
                if df3.iloc[i]['중복 횟수(10일)'] > df3.iloc[i+j]['중복 횟수(10일)']:
                    df3.loc[i, 'save_kill'] = 'save'
                    df3.loc[i+j, 'save_kill'] = 'kill'
                elif df3.iloc[i]['중복 횟수(10일)'] < df3.iloc[i+j]['중복 횟수(10일)']:
                    df3.loc[i, 'save_kill'] = 'kill'
                    df3.loc[i+j, 'save_kill'] = 'save'
                else: # 동일
                    df3.loc[i, 'save_kill'] = 'kill'
                    df3.loc[i+j, 'save_kill'] = 'kill'
    else :
        j = 1
        while i+j < df3.shape[0] :
            if (df3.iloc[i]['일자'] <= df3.iloc[i+j]['일자'] <= df3.iloc[i]['일자'] + dt.timedelta(days=40)) & (df3.iloc[i]['기업'] == df3.iloc[i+j]['기업']) :
                if df3.iloc[i]['중복 횟수(10일)'] > df3.iloc[i+j]['중복 횟수(10일)']:
                    df3.loc[i, 'save_kill'] = 'save'
                    df3.loc[i+j, 'save_kill'] = 'kill'
                elif df3.iloc[i]['중복 횟수(10일)'] < df3.iloc[i+j]['중복 횟수(10일)']:
                    df3.loc[i, 'save_kill'] = 'kill'
                    df3.loc[i+j, 'save_kill'] = 'save'
                else: # 동일
                    df3.loc[i, 'save_kill'] = 'kill'
                    df3.loc[i+j, 'save_kill'] = 'kill'
            j +=1
    if i%100 == 0:
        print(i)

df3 = df3[df3['save_kill']=='save']
print(df3.shape)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
(1344, 144)


In [9]:
df3.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_result_7.xlsx')
# result7 : 40일짜리 window 안의 사건들 제거 

In [10]:
E =['환경오염']
S= ['파업','산재','고용','임금','노동권', '독과점','공급망','상생','품질','개인정보','유해물질','광고','성범죄'] 
G = ['비리','도덕성','경영권','불공정거래','분식회계']
ESG = [E,S,G]

In [11]:
yrs = [i for i in range(2012,2022)]
yr_smp = []
for yr in yrs:
    df_yr = df3[df3['일자'].dt.year == yr]
    yr_smp.append(df_yr.shape[0])
#     print(yr, df_yr.shape[0])    

print(sum(yr_smp))

for pillar in ESG:
    df_pillar = df3[df3['ESG'].isin(pillar)]
    print(df3[df3['ESG'].isin(pillar)].shape[0])
    for yr in yrs:
        df_yr = df_pillar[df_pillar['일자'].dt.year == yr]
        yr_smp.append(df_yr.shape[0])
#         print(yr, df_yr.shape[0])  

1344
44
770
530


In [302]:
# # window = [(-5,-4),(-3,-2),(-1,1),(2,3),(4,5)]
# window = [(-5,-2),(-1,-1),(0,0),(1,1),(2,5)]
# # window = [(-5,-1),(0,0),(1,5)]
# initial = -5 
# final = 5
# period = 200 # 평균값을 계산하는 기간

In [12]:
# window = [(-5,-4),(-3,-2),(-1,1),(2,3),(4,5)]
window = [(-20,-11),(-10,-5),(-5,-2),(-1,-1),(0,0),(1,1),(2,5),(6,10),(11,20)]
initial = -20 
final = 20
period = 200 # 평균값을 계산하는 기간

In [13]:
# mean, t-test

def col_gen(ms, invs):
    col_temp = []
    for m in ms:
        for inv in invs:
            col_temp.append(m+'_'+inv)
    column = []
    for c in col_temp:
        for a,b in window:
            column.append(c+'_'+str(a)+'_'+str(b))
    return column
    

In [14]:
col_AV = col_gen(['AV_VOL', 'AV_VAL'],['TOT','IND', 'INS' ,'FOR'])
col_BSI = col_gen(['BSI'],['IND', 'INS' ,'FOR'])
col_TO = col_gen(['TO'],['TOT','IND', 'INS' ,'FOR'])

In [15]:

for col in col_AV :  
    t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
    print(col, '\n', 'mean:', round(df3[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_BSI:
    t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
    print(col, '\n', 'mean:', round(df3[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_TO:
    t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
    print(col, '\n', 'mean:', round(df3[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))

AV_VOL_TOT_-20_-11 
 mean: 0.03805 t-stats: 1.51676 p-value: 0.12956
AV_VOL_TOT_-10_-5 
 mean: 0.10531 t-stats: 3.1256 p-value: 0.00181
AV_VOL_TOT_-5_-2 
 mean: 0.14868 t-stats: 3.42492 p-value: 0.00063
AV_VOL_TOT_-1_-1 
 mean: 0.16002 t-stats: 3.27218 p-value: 0.00109
AV_VOL_TOT_0_0 
 mean: 0.39079 t-stats: 3.79648 p-value: 0.00015
AV_VOL_TOT_1_1 
 mean: 0.23808 t-stats: 4.05356 p-value: 5e-05
AV_VOL_TOT_2_5 
 mean: 0.13567 t-stats: 2.80858 p-value: 0.00505
AV_VOL_TOT_6_10 
 mean: 0.146 t-stats: 2.35129 p-value: 0.01885
AV_VOL_TOT_11_20 
 mean: 0.17544 t-stats: 3.98183 p-value: 7e-05
AV_VOL_IND_-20_-11 
 mean: 0.07777 t-stats: 2.09491 p-value: 0.03637
AV_VOL_IND_-10_-5 
 mean: 0.18485 t-stats: 3.45008 p-value: 0.00058
AV_VOL_IND_-5_-2 
 mean: 0.25596 t-stats: 3.43324 p-value: 0.00061
AV_VOL_IND_-1_-1 
 mean: 0.25526 t-stats: 4.34686 p-value: 1e-05
AV_VOL_IND_0_0 
 mean: 0.56336 t-stats: 3.88465 p-value: 0.00011
AV_VOL_IND_1_1 
 mean: 0.37398 t-stats: 4.14311 p-value: 4e-05
AV_VOL_IND_

### 엑셀로 정리

In [16]:
def col_gen_m(ms, invs):
    col_temp = []
    for m in ms:
        for inv in invs:
            col_temp.append(m+'_'+inv)
    return col_temp

In [17]:
col_m = col_gen_m(['AV_VOL', 'AV_VAL','BSI','TO'],['TOT','IND', 'INS' ,'FOR'])
col_m.remove('BSI_TOT')

In [18]:
# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        m= df3[col].mean()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(df3[col], 0)
        
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [19]:
df_result

AV_VOL_TOT        AV_VOL_IND        AV_VOL_INS        AV_VOL_FOR  \
                 mean t_stat       mean t_stat       mean t_stat       mean   
(-20, -11)     0.0381   1.52   0.0778**   2.09    -0.0245  -1.31     0.0204   
(-10, -5)    0.105***   3.13   0.185***   3.45     0.0162   0.81   0.0496**   
(-5, -2)     0.149***   3.42   0.256***   3.43   0.0563**   2.02  0.0832***   
(-1, -1)      0.16***   3.27   0.255***   4.35   0.0742**   2.08    0.112**   
(0, 0)       0.391***   3.80   0.563***   3.88   0.171***   3.22   0.317***   
(1, 1)       0.238***   4.05   0.374***   4.14   0.172***   3.43   0.124***   
(2, 5)       0.136***   2.81   0.259***   3.72     0.0458   1.64   0.0537**   
(6, 10)       0.146**   2.35   0.265***   3.14     0.0487   1.58   0.0496**   
(11, 20)     0.175***   3.98    0.32***   4.90     0.116*   1.66  0.0826***   

                  AV_VAL_TOT         ...     BSI_FOR             TO_TOT  \
           t_stat       mean t_stat  ...        mean t_stat        mean   
(-20, -11)   1.17   0.0669**   2.32  ...    -0.0046*  -1.89   0.000978*   
(-10, -5)    2.45   0.122***   3.42  ...     0.00255   0.95   0.0013***   
(-5, -2)     3.01   0.178***   3.77  ...     0.0053*   1.79  0.00116***   
(-1, -1)     2.10   0.191***   3.34  ...    0.000694   0.15   0.00172**   
(0, 0)       3.00   0.459***   3.91  ...   0.0124***   2.69   0.00339**   
(1, 1)       3.31   0.299***   4.31  ...     0.00632   1.40  0.00142***   
(2, 5)       2.22   0.195***   3.05  ...    -0.00161  -0.52  0.000722**   
(6, 10)      2.23    0.189**   2.39  ...    -0.00302  -1.08    0.000505   
(11, 20)     2.92   0.234***   3.70  ...  -0.00591**  -2.41   0.000564*   

                       TO_IND              TO_INS              TO_FOR         
           t_stat        mean t_stat         mean t_stat         mean t_stat  
(-20, -11)   1.83   0.000908*   1.76     1.08e-05   0.66  6.51e-05***   2.83  
(-10, -5)    2.79  0.00116***   2.62   4.81e-05**   2.55  0.000103***   3.62  
(-5, -2)     2.75  0.000976**   2.54  6.19e-05***   2.61  0.000118***   3.77  
(-1, -1)     2.15    0.00148*   1.94   9.28e-05**   2.27  0.000164***   3.18  
(0, 0)       2.52   0.00276**   2.17   0.00021***   3.76   0.00042***   3.13  
(1, 1)       2.71   0.00112**   2.30  0.000148***   4.13  0.000168***   3.99  
(2, 5)       2.30  0.000593**   2.09  6.61e-05***   2.78   8.3e-05***   3.24  
(6, 10)      1.64    0.000403   1.43     3.7e-05*   1.83   6.8e-05***   2.81  
(11, 20)     1.93    0.000426   1.61     4.2e-05*   1.94  8.57e-05***   3.26  

[9 rows x 30 columns]

In [20]:
df_result.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_t_test_7.xlsx')
# winsorize 1% -> 5% 로 해야 유의성 사라짐
# result_7 : 40일짜리 window 안의 사건들 제거 

overlap

In [323]:
ov = 5
# df4 = df3[df3['중복 횟수']>ov]
######################### 애초에 news가 1개 있는 기업은 살리기 -> 1년 동안 1개 -> 1분기 동안 1개...
df4 = pd.DataFrame()
firm_lst = df3['기업'].value_counts().index.to_list() #기업명 리스트

for f in firm_lst:
    df3_temp = df3[(df3['기업']==f)]
    if df3_temp.shape[0] > 1: 
        df4_temp = df3_temp[df3_temp['중복 횟수']>ov]
        df4 = pd.concat([df4, df4_temp])
        
    elif df3_temp.shape[0] ==1 : # news가 1개이면 굳이 뺄 필요없음
        df4 = pd.concat([df4, df3_temp])
df4.reset_index(inplace =True, drop=True)

In [324]:
############# overlap ##############

# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        ### winsor ####
        m= df4[col].mean()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(df4[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(df4[col], 0)
        ###############
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [325]:
df_result

AV_VOL_TOT        AV_VOL_IND        AV_VOL_INS        AV_VOL_FOR  \
               mean t_stat       mean t_stat       mean t_stat       mean   
(-5, -2)     0.0307   0.65     0.0601   1.13   -0.0539*  -1.68     0.0591   
(-1, -1)  -0.0737**  -1.97    -0.0244  -0.51  -0.0957**  -2.42   -0.0639*   
(0, 0)       0.212*   1.65    0.325**   2.22     0.0431   0.77     0.136*   
(1, 1)        0.264   1.58     0.391*   1.95     0.0589   0.57     0.0674   
(2, 5)         0.17   1.19       0.29   1.57    -0.0143  -0.30     0.0264   

                AV_VAL_TOT         ...   BSI_FOR             TO_TOT         \
         t_stat       mean t_stat  ...      mean t_stat        mean t_stat   
(-5, -2)   1.30     0.0715   1.39  ...  -0.00273  -0.50    0.000719   0.76   
(-1, -1)  -1.77   -0.00761  -0.20  ...   0.00831   1.04  -0.000685*  -1.90   
(0, 0)     1.71    0.309**   2.07  ...   0.00787   0.99    0.000999   1.04   
(1, 1)     1.07     0.394*   1.82  ...   0.00391   0.51    0.000483   0.72   
(2, 5)     0.55      0.323   1.55  ...  -0.0104*  -1.96    0.000103   0.15   

              TO_IND            TO_INS             TO_FOR         
                mean t_stat       mean t_stat        mean t_stat  
(-5, -2)    0.000611   0.71  -1.07e-05  -0.37   0.000109*   1.88  
(-1, -1)  -0.000632*  -1.94  -2.59e-05  -0.71   -1.01e-05  -0.26  
(0, 0)      0.000742   0.84  7.04e-05*   1.70  0.000175**   2.46  
(1, 1)      0.000324   0.54   2.49e-05   0.62   0.000104*   1.72  
(2, 5)      0.000105   0.17  -6.25e-06  -0.16    1.38e-05   0.29  

[5 rows x 30 columns]

## winsorize

In [36]:
############# winsorize ##############
thres = 0.01
def wins(df, col):
    df_wins = df.copy()
    df_wins.sort_values(by=[col], inplace=True)
    df_wins.reset_index(inplace=True, drop=True)
    cut = int(df_wins.shape[0]*thres)
    for i in range(cut):
        df_wins.loc[i,col] = df_wins.loc[cut,col]
        df_wins.loc[df_wins.shape[0]-1-i,col] = df_wins.loc[df_wins.shape[0]-1-cut,col]
    return df_wins

In [40]:
############# winsorize ##############
# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        ### winsor ####
        m= wins(df3,col)[col].mean()
        med = wins(df3,col)[col].median()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df3,col)[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df3,col)[col], 0)
        ###############
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리
        med=str(round_it(med,3))

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat,med])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat','median']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [43]:
df_result

AV_VOL_TOT                 AV_VOL_IND                 AV_VOL_INS  \
                 mean t_stat   median       mean t_stat  median        mean   
(-20, -11)    0.00436   0.25  -0.0885     0.0263   1.24  -0.107  -0.0439***   
(-10, -5)   0.0616***   2.90  -0.0748   0.116***   4.20  -0.117     0.00665   
(-5, -2)    0.0912***   3.65  -0.0754   0.169***   4.93  -0.134      0.0237   
(-1, -1)     0.103***   3.46   -0.124   0.183***   4.78  -0.161      0.0411   
(0, 0)       0.218***   6.35  -0.0673   0.319***   7.22  -0.108    0.109***   
(1, 1)       0.142***   4.35   -0.112   0.222***   5.20  -0.166    0.095***   
(2, 5)       0.0621**   2.53   -0.104   0.144***   4.20  -0.127      0.0184   
(6, 10)     0.0648***   2.69  -0.0939   0.126***   4.18  -0.128     0.00105   
(11, 20)     0.122***   4.05  -0.0638   0.211***   5.56  -0.103      0.0324   

                           AV_VOL_FOR  ...     TO_TOT       TO_IND         \
           t_stat   median       mean  ...     median         mean t_stat   
(-20, -11)  -2.93  -0.0782     0.0104  ...  -0.000212   0.000337**   2.44   
(-10, -5)    0.37  -0.0737    0.036**  ...  -0.000229  0.000567***   3.25   
(-5, -2)     1.17  -0.0915   0.0474**  ...  -0.000251  0.000545***   3.14   
(-1, -1)     1.48   -0.152    0.049**  ...  -0.000393  0.000555***   2.79   
(0, 0)       3.53   -0.102   0.144***  ...  -0.000317    0.0014***   4.76   
(1, 1)       3.06   -0.132  0.0698***  ...  -0.000363  0.000664***   3.36   
(2, 5)       0.83  -0.0848     0.036*  ...  -0.000293   0.000425**   2.54   
(6, 10)      0.05  -0.0909    0.0347*  ...  -0.000318     0.000237   1.60   
(11, 20)     1.48   -0.057   0.0475**  ...  -0.000247   0.000358**   1.98   

                            TO_INS                         TO_FOR         \
               median         mean t_stat     median         mean t_stat   
(-20, -11)  -0.000105     3.65e-06   0.25  -4.76e-05  5.46e-05***   2.93   
(-10, -5)   -0.000105   3.89e-05**   2.35  -4.93e-05  8.55e-05***   3.85   
(-5, -2)    -0.000112   4.08e-05**   2.22  -5.43e-05  9.53e-05***   3.96   
(-1, -1)    -0.000153    4.28e-05*   1.77  -9.59e-05  9.02e-05***   3.15   
(0, 0)      -0.000122  0.000127***   4.44  -7.81e-05  0.000227***   6.04   
(1, 1)      -0.000171  0.000112***   4.06  -7.58e-05  0.000133***   4.16   
(2, 5)      -0.000131  5.11e-05***   2.60  -6.57e-05  7.35e-05***   3.38   
(6, 10)     -0.000129     2.48e-05   1.42  -5.55e-05  6.63e-05***   3.22   
(11, 20)    -9.51e-05     1.89e-05   1.20  -3.05e-05  8.36e-05***   3.69   

                       
               median  
(-20, -11)  -4.02e-05  
(-10, -5)   -4.56e-05  
(-5, -2)    -4.44e-05  
(-1, -1)    -9.22e-05  
(0, 0)      -7.82e-05  
(1, 1)      -9.26e-05  
(2, 5)      -6.16e-05  
(6, 10)     -5.52e-05  
(11, 20)    -4.86e-05  

[9 rows x 45 columns]

In [42]:
df_result.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_t_test_7_wins{}.xlsx'.format(thres))
# winsorize 1% -> 5% 로 해야 유의성 사라짐

## 중복 횟수 0회 제거: df3 -> df4 (info power 가진 event 선정)

In [333]:
firm_lst = df3['기업'].value_counts().index.to_list() #기업명 리스트

for f in firm_lst:
    df3_temp = df3[(df3['기업']==f)]
    print(f,round(df3_temp['중복 횟수'].mean(),3), '\t', round(df3_temp['중복 횟수'].std(),3))

현대차 1.813 	 3.627
기아 1.144 	 2.246
삼성전자 1.593 	 3.402
강원랜드 3.224 	 6.682
KT 4.127 	 7.332
대한항공 2.266 	 5.456
쌍용차 0.574 	 1.485
대우건설 1.514 	 2.942
이마트 1.595 	 3.198
금호타이어 1.807 	 3.602
KT&G 3.009 	 4.42
대우조선해양 0.944 	 2.004
현대제철 0.75 	 1.665
KB금융 0.825 	 1.605
LG전자 1.357 	 2.227
현대건설 1.283 	 2.491
롯데케미칼 1.959 	 2.708
SK텔레콤 0.347 	 0.879
SK하이닉스 2.068 	 5.475
LG화학 1.628 	 3.125
남양유업 2.023 	 3.751
한미약품 1.595 	 2.328
롯데쇼핑 1.925 	 2.823
CJ대한통운 1.105 	 2.252
GS건설 0.921 	 1.617
삼성중공업 0.892 	 1.712
영풍 0.568 	 1.819
한진칼 0.771 	 2.276
고려아연 1.412 	 3.759
한샘 1.515 	 2.502
삼성바이오로직스 1.879 	 3.489
기업은행 0.469 	 0.983
삼성물산 1.034 	 3.53
동국제강 2.379 	 4.917
카카오 1.0 	 1.363
금호석유 0.778 	 2.044
태광산업 0.462 	 1.067
하이트진로 1.04 	 1.859
삼성증권 2.28 	 4.287
한국가스공사 0.583 	 1.349
코웨이 0.696 	 1.893
HMM 2.522 	 4.337
SK네트웍스 6.182 	 10.568
한국전력 0.091 	 0.294
한전기술 0.545 	 1.101
농심 1.773 	 3.294
현대글로비스 0.524 	 1.569
삼성SDI 1.0 	 1.453
엔씨소프트 1.556 	 3.552
삼양식품 3.333 	 7.054
BNK금융지주 1.176 	 1.704
LG디스플레이 0.588 	 1.064
CJ제일제당 1

In [287]:
ov = 19
df4 = df3[df3['중복 횟수']>ov]

In [288]:
######################### 애초에 news가 1개 있는 기업은 살리기 -> 1년 동안 1개 -> 1분기 동안 1개...
df4 = pd.DataFrame()
firm_lst = df3['기업'].value_counts().index.to_list() #기업명 리스트

for f in firm_lst:
    df3_temp = df3[(df3['기업']==f)]
    if df3_temp.shape[0] > 1: 
        df4_temp = df3_temp[df3_temp['중복 횟수']>ov]
        df4 = pd.concat([df4, df4_temp])
        
    elif df3_temp.shape[0] ==1 : # news가 1개이면 굳이 뺄 필요없음
        df4 = pd.concat([df4, df3_temp])


In [289]:
df4.reset_index(inplace =True, drop=True)

In [290]:
print(df4)
print(df4.shape)

           일자                                  제목      기업    ESG  중복 횟수  \
0  2016-07-05            현대차 임협 결렬 선언...파업 절차 밟기로     현대차     파업     26   
1  2016-09-26   현대차 노조 12년 만에 전면파업..."자동차 산업도 위기"     현대차     파업     23   
2  2016-10-05           與 '현대차 노조 파업중단 촉구' 결의안 추진     현대차     파업     20   
3  2018-07-05       검찰, 공정위 취업특혜 의혹 현대차ㆍ쿠팡 등 압수수색     현대차     비리     20   
4  2019-07-09      '불법 파견' 기아차 사장 기소...노조 "반쪽 수사"      기아     고용     21   
..        ...                                 ...     ...    ...    ...   
89 2021-05-04              검찰, ‘쌍방울 주가조작’ 주범 구속기소     쌍방울  불공정거래      1   
90 2017-07-20  포항 세아특수강서 염산 누출...직원 2명 화상 관리부실 조사   세아특수강     산재      0   
91 2020-01-23       노조설립 직원 성추행 해고 삼영전자, 항소심에서 패소    삼영전자    노동권      0   
92 2016-12-22          다이옥신 배출 의혹 삼양사, 폐기물도 불법 매립     삼양사   환경오염      0   
93 2021-04-16     효성첨단소재 공장서 황산 1천ℓ 누출 “인명 피해 없어”  효성첨단소재     산재      1   

   save_kill     제거  AV_VOL_TOT_-5_-2  AV_VOL_TOT_-1_-1  AV_VOL_TOT_0_0  ...  \
0       save  False

In [291]:
df4['기업'].value_counts().shape

(69,)

In [292]:
############# winsorize
for col in col_AV :  
    t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
    print(col, '\n', 'mean:', round(wins(df4,col)[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_BSI:
    t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
    print(col, '\n', 'mean:', round(wins(df4,col)[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))
print('\n')
for col in col_TO:
    t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
    print(col, '\n', 'mean:', round(wins(df4,col)[col].mean(),5), 't-stats:', round(t_stat,5), 'p-value:', round(p_value,5))

AV_VOL_TOT_-5_-2 
 mean: 0.17283 t-stats: 0.98687 p-value: 0.32627
AV_VOL_TOT_-1_-1 
 mean: -0.10186 t-stats: -1.22852 p-value: 0.22235
AV_VOL_TOT_0_0 
 mean: 0.0914 t-stats: 0.94336 p-value: 0.34794
AV_VOL_TOT_1_1 
 mean: 1.01516 t-stats: 1.3902 p-value: 0.16779
AV_VOL_TOT_2_5 
 mean: 0.84013 t-stats: 1.33946 p-value: 0.18369
AV_VOL_IND_-5_-2 
 mean: 0.19887 t-stats: 1.09213 p-value: 0.2776
AV_VOL_IND_-1_-1 
 mean: -0.08385 t-stats: -0.74405 p-value: 0.45872
AV_VOL_IND_0_0 
 mean: 0.11333 t-stats: 1.06973 p-value: 0.28751
AV_VOL_IND_1_1 
 mean: 1.19146 t-stats: 1.36659 p-value: 0.17505
AV_VOL_IND_2_5 
 mean: 1.11186 t-stats: 1.39919 p-value: 0.16508
AV_VOL_INS_-5_-2 
 mean: -0.01323 t-stats: -0.13671 p-value: 0.89156
AV_VOL_INS_-1_-1 
 mean: -0.02975 t-stats: -0.25978 p-value: 0.79561
AV_VOL_INS_0_0 
 mean: 0.07053 t-stats: 0.42506 p-value: 0.67178
AV_VOL_INS_1_1 
 mean: 0.44692 t-stats: 1.01672 p-value: 0.31192
AV_VOL_INS_2_5 
 mean: 0.16495 t-stats: 0.97476 p-value: 0.33221
AV_VOL_F

In [293]:
############# overlap ##############
############# winsorize ##############

# 유효숫자 함수
def round_it(x, sig):
    return round(x, sig-int(floor(log10(abs(x))))-1)

df_result = []
for i in col_m: 
    ms_win = [] # AV_TOT_-5_0 등을 담는 list
    for a, b in window:
        ms_win.append(i+'_'+str(a)+'_'+str(b))
    
    ms_win_result = []
    for col in ms_win: 
        ### winsor ####
        m= wins(df4,col)[col].mean()
        if 'AV' in i: # Ab. vol.이면 t-test 대상은 1
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0) #################### 0인지 1인지 ####################
        else: # 그외 0
            t_stat, p_value = scipy.stats.ttest_1samp(wins(df4,col)[col], 0)
        ###############
        
        m=str(round_it(m,3))  # 유효숫자 3자리
        t_stat = '%.2f' % t_stat  # 소수점 둘째 자리

        if 0.05 <= p_value < 0.1:
            m = m+'*'
        elif 0.01 <= p_value < 0.05:
            m = m+'**'
        elif p_value < 0.01:
            m = m+'***'

        ms_win_result.append([m,t_stat])
    
    ms_win_result = np.array(ms_win_result)
    ms_win_result = np.transpose(ms_win_result)
    
    if i == col_m[0]:
        df_result = ms_win_result
    else:
        df_result = np.vstack((df_result,ms_win_result))
    

subcol = ['mean','t_stat']

column = pd.MultiIndex.from_product([col_m, subcol])
df_result = np.transpose(df_result)
df_result = pd.DataFrame(df_result, index = window, columns = column)

In [294]:
## 별도의 처리X
# df_result.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_t_test_5_ov{}_wins{}.xlsx'.format(ov,thres))

In [295]:
######################### 애초에 news가 1개 있는 기업은 살리기 save
df_result.to_excel('C:/Users/Jeongseok_Bang/OneDrive/바탕 화면/석사과정/FinLab/ESG/DF_최종_10_AV_t_test_5_ov{}_wins{}_addsingle.xlsx'.format(ov,thres))